<a href="https://colab.research.google.com/github/natsakh/IAD/blob/main/Pr_7/7_1_Transformer_Encoder%E2%80%93Decoder_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import torch
import torch.nn as nn

#Transformer Encoder

In [24]:
x = torch.rand(32, 50, 64)
# batch=32, seq_len=50, features=d_model=64

layer = nn.TransformerEncoderLayer(
    d_model=64,
    nhead=8,
    dim_feedforward=256,
    batch_first=True
)

out = layer(x)   # x shape: [32, 50, 64]

print("Input:", x.shape)
print("Output:", out.shape)


Input: torch.Size([32, 50, 64])
Output: torch.Size([32, 50, 64])


In [25]:
encoder_layer = nn.TransformerEncoderLayer(
    d_model=64, nhead=8, dim_feedforward=256, batch_first=True
)
encoder = nn.TransformerEncoder(encoder_layer, num_layers=4)

out = encoder(x)

print("Input:", x.shape)
print("Output:", out.shape)


Input: torch.Size([32, 50, 64])
Output: torch.Size([32, 50, 64])


#Decoder only

In [26]:
import torch
import torch.nn as nn

# batch=32, seq_len=20, d_model=64
x = torch.rand(32, 20, 64)   # embeddings
print("Input x shape:", x.shape)    # [32, 20, 64]

# Один шар декодера трансформера
decoder_layer = nn.TransformerDecoderLayer(
    d_model=64,
    nhead=8,
    dim_feedforward=256,
    batch_first=True,
)

# Стек із кількох шарів
decoder = nn.TransformerDecoder(
    decoder_layer,
    num_layers=3
)

# Каузальна маска: кожна позиція "бачить" лише попередні
B, T, D = x.shape
tgt_mask = torch.triu(torch.ones(T, T, dtype=torch.bool), diagonal=1) # True = заборонено
#TransformerDecoderLayer приймає дві різні форми масок:
#bool mask (True = заборонено)
#float mask (-inf = заборонено) mask = torch.triu(torch.ones(T, T) * float('-inf'), diagonal=1)

# decoder-only: tgt = emb, memory = emb
emb = x

out = decoder(
    tgt=emb,          # [B, T, D]
    memory=emb,       # [B, T, D] – те саме, що і tgt
    tgt_mask=tgt_mask
)

print("tgt (emb) shape   :", emb.shape)
print("memory shape      :", emb.shape)
print("decoder out shape :", out.shape)


Input x shape: torch.Size([32, 20, 64])
tgt (emb) shape   : torch.Size([32, 20, 64])
memory shape      : torch.Size([32, 20, 64])
decoder out shape : torch.Size([32, 20, 64])


In [27]:
#маска
T = 6
tgt_mask = torch.triu(torch.ones(T, T, dtype=torch.bool), diagonal=1)
print(tgt_mask)

tensor([[False,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False, False, False]])


#Encoder Decoder Transformer

In [28]:
B = 32          # batch size
S = 10          # довжина вхідної послідовності (source)
T = 12          # довжина вихідної послідовності (target)
D = 64          # розмір ембедінга (d_model)

# ембедінги для src і tgt
src = torch.rand(B, S, D)   # [B, S, D]
tgt = torch.rand(B, T, D)   # [B, T, D]

print("src shape:", src.shape)
print("tgt shape:", tgt.shape)

# ---- Encoder ----
encoder_layer = nn.TransformerEncoderLayer(
    d_model=D,
    nhead=8,
    dim_feedforward=256,
    batch_first=True,
)
encoder = nn.TransformerEncoder(
    encoder_layer,
    num_layers=2
)

# ---- Decoder ----
decoder_layer = nn.TransformerDecoderLayer(
    d_model=D,
    nhead=8,
    dim_feedforward=256,
    batch_first=True,
)
decoder = nn.TransformerDecoder(
    decoder_layer,
    num_layers=2
)

# Каузальна маска для декодера: кожна позиція "бачить" лише попередні
def generate_subsequent_mask(size, device):
    # True = заборонено, False = можна дивитись
    mask = torch.triu(
        torch.ones(size, size, dtype=torch.bool, device=device),
        diagonal=1
    )
    return mask

tgt_mask = generate_subsequent_mask(T, tgt.device)   # [T, T]

# ---- Прямий прохід через Encoder + Decoder ----

# 1) Пропускаємо src через енкодер → отримуємо memory
memory = encoder(src)        # [B, S, D]

# 2) Декодер дивиться на tgt (з каузальною маскою) + memory
out = decoder(
    tgt=tgt,                 # [B, T, D]
    memory=memory,           # [B, S, D]
    tgt_mask=tgt_mask        # [T, T]
)

print("memory (encoder out) shape:", memory.shape)
print("decoder out shape         :", out.shape)


src shape: torch.Size([32, 10, 64])
tgt shape: torch.Size([32, 12, 64])
memory (encoder out) shape: torch.Size([32, 10, 64])
decoder out shape         : torch.Size([32, 12, 64])
